# Random Signals and LTI-Systems

*This jupyter notebook is part of a [collection of notebooks](../index.ipynb) on various topics of Digital Signal Processing. Please direct questions and suggestions to [Sascha.Spors@uni-rostock.de](mailto:Sascha.Spors@uni-rostock.de).*

## Linear Prediction

In general, [predictive modeling](https://en.wikipedia.org/wiki/Predictive_modelling) aims at predicting outcomes using (statistical) models of the underlying processes. When applied to discrete signals, it refers to predicting future values from past ones. Different signal models are used for this purpose. [Linear prediction](https://en.wikipedia.org/wiki/Linear_prediction) aims at predicting the actual value $x[k]$ of a discrete random signal by a linear combination of its past values. It is frequently applied in the lossless coding of signals. For instance, [linear predictive coding](https://en.wikipedia.org/wiki/Linear_predictive_coding) (LPC) is a major building block of many [speech coding](https://en.wikipedia.org/wiki/Speech_coding) and speech synthesis techniques.

### Theory

It is assumed that $x[k]$ is a wide-sense stationary (WSS) random signal. The prediction $\hat{x}[k]$ of the current sample $x[k]$ is computed from a weighted superposition of the past $N$ samples

\begin{equation}
\hat{x}[k] = \sum_{\kappa = 1}^{N} h_\kappa[k] \, x[k - \kappa]
\end{equation}

where $N$ denotes the order of the predictor and $h_\kappa[k]$ the weights of the past $N$ samples $x[k - \kappa]$ at time-instant $k$. The weights might differ for different time-instants. This constitutes a [moving average](https://en.wikipedia.org/wiki/Moving-average_model) (MA) model or a linear filter with finite impulse response (FIR). Therefore the weights $h_\kappa[k]$ can be interpreted as the filter coefficients of a time-variant non-recursive filter with FIR.

It is beneficial to express above sum as a product of vectors. Combining the $N$ filter coefficients and samples of the signal into the $N \times 1$ vectors

\begin{align}
\mathbf{x}[k] &= [ x[k], x[k-1], \dots, x[k-N+1] ]^T \\
\mathbf{h}[k] &= [ h_1[k], h_2[k], \dots, h_N[k] ]^T
\end{align}

yields

\begin{equation}
\hat{x}[k] = \mathbf{h}^T[k] \, \mathbf{x}[k-1]
\end{equation}

The aim of linear prediction is to determine the filter coefficients such that the predicted signal $\hat{x}[k]$ matches the signal $x[k]$ as close as possible. In order to quantify the deviation between the signal $x[k]$ and its predicted value $\hat{x}[k]$ at the time-instance $k$, the error signal

\begin{align}
e[k] &= x[k] - \hat{x}[k] \\
&= x[k] - \mathbf{h}^T[k] \, \mathbf{x}[k-1]
\end{align}

is introduced. The error $e[k]$ equals zero, if the signal $x[k]$ is perfectly predicted. In general, this cannot be achieved by a finite predictor order $N$. The problem of determining suitable filter coefficients could be approached by directly minimizing the (average) error. However, this constitutes a non-convex optimization problem. Instead, the quadratic average of the error $e[k]$ is typically used. This measure is known as [*mean squared error*](https://en.wikipedia.org/wiki/Mean_squared_error) (MSE). It is defined as

\begin{align}
J &= E \{ | e[k] |^2 \} \\
&= E \{ | x[k] - \mathbf{h}^T[k] \, \mathbf{x}[k-1] |^2 \}
\end{align}

Above equation is referred to as [*cost function*](https://en.wikipedia.org/wiki/Loss_function) $J$ of the optimization problem. We aim at minimizing the cost function, hence minimizing the MSE between the signal $x[k]$ and its prediction $\hat{x}[k]$. The solution is referred to as [minimum mean squared error](https://en.wikipedia.org/wiki/Minimum_mean_square_error) (MMSE) solution. Minimizing the cost function is achieved by calculating its gradient with respect to the filter coefficients

\begin{align}
\nabla_\mathbf{h} J &= -2 E \left\{ x[k-1] (x[k] - \mathbf{h}^T[k] \mathbf{x}[k-1]) \right\} \\
&= - 2 \mathbf{r}[k] + 2 \mathbf{R}[k] \mathbf{h}[k]
\end{align}

where

\begin{align}
\mathbf{R}[k] &= E \{ \mathbf{x}[k-1] \mathbf{x}^T[k-1] \} \\
\mathbf{r}[k] &= E \{ \mathbf{x}[k-1] x[k] \}
\end{align}

denote the auto-correlation matrix and cross-correlation vector, respectively. As the MMSE solution constitutes a [convex optimization](https://en.wikipedia.org/wiki/Convex_optimization) problem, the minimum of the cost function is found by setting its gradient to zero, $J = 0$. Rearranging terms this results in the *Wiener-Hopf* equation

\begin{equation}
\mathbf{R}[k] \mathbf{h}[k] = \mathbf{r}[k] 
\end{equation}

The optimum filter coefficients in the MMSE sense are then given as

\begin{equation}
\mathbf{h}[k] = \mathbf{R}^{-1}[k] \mathbf{r}[k] 
\end{equation}

### Example - Linear Prediction of a Speech Signal

Recorded speech signals are in general not WSS random signals. However, the stationary part of a vocal can be considered reasonably well as WSS. The following example illustrates linear prediction using a recording of the vocal 'o'. 

#### Computation of prediction filter

First the signal is loaded, conditioned and truncated

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

L = 32768  # total length of signal
N = 8  # length of predictor

# read and truncate audio file
fs, x = wavfile.read('../data/vocal_o_8k.wav')
x = np.asarray(x, dtype=float)/2**15
x = x[:L]

As we only have access to one recorded instance of the vocal, the auto-correlation matrix and the cross-correlation vector are estimated by averaging with respect to time. For instance

\begin{equation}
\mathbf{R} \approx \frac{1}{L-N+1} \sum_{k = N}^{L} \mathbf{x}[k-1] \mathbf{x}^T[k-1]
\end{equation}

where $L$ denotes the total number of samples in $x[k]$.

In [ ]:
R = np.zeros(shape=(N, N))
r = np.zeros(shape=(N,1))

for k in np.arange(N, L):
    xk = np.expand_dims(np.flip(x[k-N:k]), 1)
    R += 1/(L+2-N) * xk * xk.T
    r += 1/(L+2-N) * xk * x[k]

Lets inspect the estimated auto-correlation matrix by plotting its values

In [ ]:
plt.figure(figsize=(3, 3))
plt.matshow(R)
plt.title(r'Estimate of auto-correlation matrix $\mathbf{R}$')
plt.colorbar(fraction=0.045)

As expected, the elements on the main diagonal have the largest values. The values decay with an increasing distance of the minor diagonals. Hence, the statistical dependencies between samples at two different time instants of the signal $x[k]$ decrease with an increasing distance between the samples. These statistical dependencies are exploited for the prediction of the current signal value from past ones.

Now the filter coefficients are computed and plotted

In [ ]:
h = np.linalg.inv(R) @ r
h = np.squeeze(h)

plt.figure(figsize=(8, 3))
plt.stem(h, basefmt='C0:')
plt.xlabel(r'$k$')
plt.ylabel(r'$h_k$')
plt.grid()

#### Forward prediction

The value of the signal $x[k]$ at time-instant $k$ is predicted from its past $N$ values by evaluating the equation given above. This constitutes essentially a linear convolution $\hat{x}[k] = x[k] \ast \hat{h}[k]$ with the prediction filter

\begin{equation}
\hat{h}[k] = \begin{cases} 0 & \text{for } k=0 \\ h_k & \text{for } 0 < k < N \end{cases}
\end{equation}

The predicted signal $\hat{x}[k]$ is computed for all time instants $k$ and plotted in comparison with the signal $x[k]$ in order to evaluate the predictor.

In [ ]:
hp = np.concatenate(([0],h))
xp = np.convolve(x, hp)[:L]

plt.figure(figsize=(8, 4))
plt.plot(xp, label=r'predicted signal $\hat{x}[k]$')
plt.plot(x, label=r'signal $x[k]$', linestyle='--')
plt.xlabel(r'$k$')
plt.axis([10000, 10500,-.6,.6])
plt.legend()
plt.grid()

The error $e[k]$ is computed and plotted for inspection, as the difference between the actual signal and its prediction is quite low.

In [ ]:
e = x - xp

plt.figure(figsize=(8, 4))
plt.plot(e)
plt.xlabel(r'$k$')
plt.ylabel(r'$e[k]$')
plt.axis([10000, 10500, -.015, .015])
plt.grid()

#### Prediction from residual error

The error signal may be rewritten as

\begin{align}
e[k] &= x[k] - \mathbf{h}^T[k] \, \mathbf{x}[k-1] \\
&= h_e[k] \ast x[k]
\end{align}

where

\begin{equation}
h_e[k] = \begin{cases} 1 & \text{for } k=0 \\ - h_k & \text{for } 0 < k < N \end{cases}
\end{equation}

denotes the prediction-error filter. Taking the $z$-transform of the error signal yields

\begin{equation}
E(z) = H_e(z) \cdot X(z)
\end{equation}

where for instance $E(z) = \mathcal{Z} \{ e[k] \}$. From this result it can be concluded that the signal $x[k]$ can be computed from the error signal $e[k]$ by inverse filtering with the prediction-error filter

\begin{equation}
x[k] = \underbrace{\mathcal{Z} \{ \frac{1}{H_e(z)} \}}_{h_e^{-1}[k]} \ast e[k]
\end{equation}

The inverse of the prediction-error filter $h_e^{-1}[k]$ constitutes an only-pole (recursive) filter as the prediction filter is a zero-only (non-recursive) filter. In the following, the signal $x[k]$ is computed from the error signal $e[k]$ and plotted.

In [ ]:
from scipy.signal import lfilter

he = np.concatenate(([1],-h))
xe = lfilter([1], he, e)

plt.figure(figsize=(8, 4))
plt.plot(xe)
plt.xlabel(r'$k$')
plt.ylabel(r'reconstructed signal $x[k]$')
plt.axis([10000, 10500,-.6,.6])
plt.grid()

#### Application - Speech synthesis using an artificial glottal signal

Linear prediction is used as [linear predictive coding](https://en.wikipedia.org/wiki/Linear_predictive_coding) (LPC) in the coding and synthesis of speech signals. The latter application is briefly presented in the sequel. It bases on the [source-filter model](https://en.wikipedia.org/wiki/Source%E2%80%93filter_model) of human speech production. Here speech production is modeled as a sound source filtered by a linear acoustic filter. In the case of vocals, this sound source are the [vocal cords](https://en.wikipedia.org/wiki/Vocal_cords) which generate the periodic glottal signal. The [vocal tract](https://en.wikipedia.org/wiki/Vocal_tract) may be interpreted as an acoustic filter. Vocals are articulated by changing the air flow through the vocal cords, their state as well as the configuration of the vocal tract. Due to its acoustical properties, the vocal tract can be represented as an only-pole filter.

The basic idea of LPC-based speech synthesis is that the inverse of the prediction-error filter is an only-pole filter. When interpreting the error signal $e[k]$ as glottal signal in the source-filter model, the inverse of the prediction-error filter can be used to generate the speech signal $x[k]$. A synthetic vocal may be generated by using a synthetic glottal signal. A simple model for the glottal signal is a mean-free periodic rectangular signal with a duty cycle of 1:10. The signal is generated and plotted for illustration.

In [ ]:
from scipy.signal import square


def glottal_signal(f0):
    '''Generates a synthetic glottal signal with fundamental frequency f0 in Hertz.'''
    w0 = 2 * np.pi * f0
    t = 1/fs * np.arange(L)
    g = 1/2 * (1 + square(w0*t, duty=1/10))

    return - g + 0.1


g = 0.0055 * glottal_signal(100)

plt.figure(figsize=(8, 4))
plt.stem(g[:100])
plt.xlabel(r'$k$')
plt.ylabel(r'glottal signal $e[k]$')
plt.grid()

The spectral contents of the error signal $e[k]$ and the synthetic glottal signal $g[k]$ are compared by estimating their power spectral densities (PSDs) using the [Welch technique](../spectral_estimation_random_signals/welch_method.ipynb).

In [ ]:
from scipy.signal import welch

f, Pee = welch(e, fs, nperseg=1024)
f, Pgg = welch(g, fs, nperseg=1024)
plt.plot(f, Pee, label=r'$\hat{\Phi}_{ee}(e^{j \Omega})$ error signal')
plt.plot(f, Pgg, label=r'$\hat{\Phi}_{gg}(e^{j \Omega})$ synthetic glottal signal')

plt.xlim([0, 1000])
plt.xlabel('frequency in Hertz')
plt.legend()
plt.grid()

The spectra of both signals constitute line spectra due to their periodicity. However, the levels of the spectral lines differ. This becomes most prominent around 600 Hertz. The synthetic glottal signal is now filtered with the inverse prediction-error filter in order to generate the synthetic speech signal.

In [ ]:
xa = lfilter([1], np.concatenate(([1],-h)), g)

plt.figure(figsize=(8, 4))
plt.plot(xa)
plt.xlabel(r'$k$')
plt.ylabel(r'reconstructed signal $x[k]$')
plt.axis([10000, 10500,-1,1])
plt.grid()

Visual comparison with the recorded speech signal shown yields that the coarse structure of both is quite similar. The differences are a consequence of the simple glottal signal used. Both signals may also be compared aurally by listening to them.

In [ ]:
wavfile.write('artificial_vocal.wav', fs, np.int16(xa*16384))

**Recorded vocal**

<audio src="./../data/vocal_o_8k.wav" controls>Your browser does not support the audio element.</audio>[../data/vocal_o_8k.wav](../data/vocal_o_8k.wav)

**Synthesized vocal**

<audio src="./artificial_vocal.wav" controls>Your browser does not support the audio element.</audio>[./artificial_vocal.wav](./artificial_vocal.wav)

**Copyright**

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebook for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Sascha Spors, Digital Signal Processing - Lecture notes featuring computational examples*.